In [ ]:
# Este notebook possui instruções importantes, mas não recomendo executá-lo localmente porque o script que faz o download da dataset consome muita memória RAM, o que pode fazer o processo Python travar em computadores com pouca memória.
# Por causa disso, criei uma cópia simplificada deste notebook no Google Colab, que pode ser acessada pelo link abaixo:
# https://colab.research.google.com/drive/1N8SINqJgnPaD1oddg49O1xyy6BkoI7Op#scrollTo=F4ZK6FEXdejz

# YOLOv4 - Como criar sua própria base de imagens

O nosso conjunto de imagens customizado para fazer o treinamento com o YOLO deve conter:

* Imagens dos objetos que queremos reconhecer, já catalogadas (com os arquivos annotation)
* Arquivos obj.data e obj.names
* Arquivo .cfg customizado
* Arquivo train.txt (test.txt é opcional)

Há duas maneiras principais para reunir as imagens
* Baixar de um dataset ou repositório como o Open Images Dataset, um dataset da Google que disponibiliza imagens para mais de 600 classes diferentes.
A maneira mais prática atualmente para baixar as imagens do Open Images Dataset é usar a ferramenta [OIDv4 Toolkit](https://github.com/EscVM/OIDv4_ToolKit).
* A segunda maneira consiste em baixar manualmente as imagens do objeto e catalogá-las utilizando alguma ferramenta de anotação (annotation tool) para obter os txt com as anotações. Esse é um processo manual e que pode ser bastante demorado, portanto recomendamos verificar se a classe escolhida não está no Dataset do Google. Se você não encontrou no Open Images Dataset o objeto que deseja detectar então é necessário baixar manualmente. Por ser um método mais demorado e cansativo nós recomendamos fazer somente caso não tiver outra maneira de conseguir facilmente baixar essa imagens.

# Coletando e catalogando as imagens do objeto para treinamento

## Etapa 1 - Clonando o repositório da ferramenta

In [ ]:
!git clone https://github.com/EscVM/OIDv4_ToolKit.git

## Etapa 2 - Acessando o diretório da ferramenta

In [ ]:
ls

In [ ]:
cd OIDv4_ToolKit/

In [ ]:
ls

## Etapa 3 - Instalando todas as bibliotecas necessárias

In [ ]:
!pip3 install -r requirements.txt

## Etapa 4 - Fazendo o download das imagens

### Conjunto de imagens de Treinamento

- https://storage.googleapis.com/openimages/web/index.html

In [ ]:
!python main.py downloader --classes Apple Coffee_cup Horse --type_csv train --limit 500 --multiclasses 1

### Conjunto de imagens de Validação

In [ ]:
!python main.py downloader --classes Apple Coffee_cup Horse --type_csv test --limit 100 --multiclasses 1

## Etapa 5 - Convertendo os arquivos de anotação

### 1. Colocar as classes no arquivo classes.txt

In [ ]:
!cat classes.txt

In [ ]:
!echo -e 'Apple\nCoffee cup\nHorse' > classes.txt

### 2. Baixar o arquivo converter_annotations.py e adicione-o ao diretório

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
!unzip /content/gdrive/My\ Drive/Cursos\ -\ recursos/YOLO/recursos/TreinamentoYOLO.zip -d /content/

In [ ]:
ls

In [ ]:
!cp /content/TreinamentoYOLO/converter_annotations.py ./

### 3. Executar o arquivo de conversão



In [ ]:
!python converter_annotations.py

## Etapa 6 - Compactar o dataset

In [ ]:
cd OID/Dataset/train/

In [ ]:
ls

In [ ]:
# salve como dataset-train
!zip -r ../../../obj.zip obj -x obj/Label/*

In [ ]:
ls

In [ ]:
cd ../../../../

In [ ]:
ls

In [ ]:
cd OIDv4_ToolKit/

In [ ]:
!cp ./obj.zip /content/gdrive/My\ Drive/Cursos\ -\ recursos/YOLO/recursos

In [ ]:
ls

In [ ]:
# salve como dataset-test
!zip -r ../../../../valid.zip valid -x valid/Label/*

In [ ]:
cd /content/

In [ ]:
ls

In [ ]:
!cp ./valid.zip /content/gdrive/My\ Drive/Cursos\ -\ recursos/YOLO/recursos

# Editando os arquivos de configuração necessários para o treinamento

In [ ]:
ls

In [ ]:
pwd

In [ ]:
!git clone https://github.com/AlexeyAB/darknet

In [ ]:
cd darknet/

In [ ]:
!make

## Etapa 7 - Definindo os arquivos de configuração

### Modificações no .cfg

In [ ]:
!cp cfg/yolov4.cfg /content/gdrive/My\ Drive/Cursos\ -\ recursos/YOLO/recursos/yolov4_custom.cfg

A seguir uma explicação dos parâmetros modificados no arquivo de configuração de treinamento da rede YOLO

**[yolo]**
- `classes` (número de classes escolhidas para montar este dataset) define o número de classes a serem classificadas

**[net]**
- `subdivisions=64` (64) indica o consumo de memória da rede: diminuir esse parâmetro vai diminuir esse consumo.
- `width=608` e `height=608` definem a resolução da imagem que vai ser lida pela rede: quanto menor, maior perfomance
- `learning_rate=0.0013` indica a taxa de aprendizagem: este valor fica entre 0 e 1, mas normalmente o colocamos abaixo de 0.1
- `max_batches=2000*classes` indica o número máximo de imagens utilizadas em um treino
- `steps=80% de max_batches, 90% de max_batches` indica a quantidade de épocas de treinamento da rede neural

**[convolutional]**
- `filters=(classes+5)*3` define quantos mapas de características a camada convolucional vai produzir na saída

**OBSERVAÇÃO.** As modificações citadas em **[yolo]** e **[convolutional]** devem ser feitas em TODAS as ocorrências dessas camadas no arquivo .cfg

### Modificações no obj.names e obj.data

In [ ]:
!touch obj.names
!touch obj.data

In [ ]:
!cp obj.names /content/gdrive/My\ Drive/Cursos\ -\ recursos/YOLO/recursos/obj.names
!cp obj.data /content/gdrive/My\ Drive/Cursos\ -\ recursos/YOLO/recursos/obj.data

A seguir uma explicação das entradas criadas nos arquivos .names e .data

**obj.names**
- Modifique para conter a listagem das classes presentes no dataset, seguindo a convenção do OID
- Cada classe está em uma linha, como mostrado a seguir
- A ordem como as classes aparecem nesse arquivo importa: Apple (0), Horse(1) e Keyboard(2). Então, quando a rede fazer detecções em uma imagem, ela vai retornar os percentuais de classificação para cada uma das classes na forma de índices.

```yaml
Apple
Horse
Keyboard
```

**obj.data**
- Modifique para conter os recursos para o treinamento e teste da rede neural, conforme o template a seguir

```yaml
# Numero de classes presentes no dataset
classes = N
# Caminho para os arquivos do dataset de treino e de validacao
train  = data/train.txt
valid  = data/test.txt
# Caminho para o arquivo .names
names = data/obj.names
# Backup do treinamento: a cada 100 epocas do treinamento, o estado da rede eh salvo nesse diretorio
backup = data/backup/
```



## Etapa 8 - Gerando o arquivo train.txt e test.txt

In [ ]:
pwd

In [ ]:
cd ..

In [ ]:
cd OIDv4_ToolKit/

In [ ]:
!unzip obj.zip -d ./data

In [ ]:
!unzip /content/valid.zip -d ./data

In [ ]:
pwd

In [ ]:
!python /content/TreinamentoYOLO/gerar_train.py

In [ ]:
!python /content/TreinamentoYOLO/gerar_test.py

In [ ]:
cd data

In [ ]:
pwd

In [ ]:
!cp train.txt /content/gdrive/My\ Drive/Cursos\ -\ recursos/YOLO/recursos/train.txt

In [ ]:
!cp test.txt /content/gdrive/My\ Drive/Cursos\ -\ recursos/YOLO/recursos/test.txt